In [2]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, reset_output
from bokeh.models import HoverTool
from bokeh.models.annotations import Label, LabelSet
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.models import CategoricalColorMapper, ColorBar
from bokeh.transform import transform
from bokeh.models import LinearInterpolator
from bokeh.palettes import Viridis256
import ipywidgets 
from bokeh.io import push_notebook
from bokeh.models import ColumnDataSource, Select
from bokeh.io import output_file, show
from bokeh.models import Dropdown, Slider
from bokeh.layouts import column, row
from bokeh.models.callbacks import CustomJS
from bokeh.io import curdoc



In [3]:
test_data = pd.read_csv("C:/Users/boyan/Desktop/tue/2IOA0/test/metro_data.csv", encoding = "latin1", sep = ";")
test_data.head()

,Timestamp,StimuliName,FixationIndex,FixationDuration,MappedFixationPointX,MappedFixationPointY,user,description
0,2586,01_Antwerpen_S1.jpg,9,250,1151,458,p1,color
1,2836,01_Antwerpen_S1.jpg,10,150,1371,316,p1,color
2,2986,01_Antwerpen_S1.jpg,11,283,1342,287,p1,color
3,3269,01_Antwerpen_S1.jpg,12,433,762,303,p1,color
4,3702,01_Antwerpen_S1.jpg,13,183,624,297,p1,color


In [4]:
# use only the 01_antwerpen_01 map


In [5]:
source = ColumnDataSource(data=dict(Timestamp=[], StimuliName=[], FixationIndex=[], FixationDuration=[], MappedFixationPointX = [], MappedFixationPointY = [], user = []))



# add colours to the scatter points
color_mapper = CategoricalColorMapper ( 
    factors = list(test_data.user.unique()),
    palette = Viridis256
)




# set size of the dots in the future scatter plot
size_mapper = LinearInterpolator (
    x = [test_data.FixationDuration.min(), test_data.FixationDuration.max()],
    y = [10, 50]
)





In [6]:
users = test_data['user'].unique().tolist()
users = sorted(users, key=lambda x: int("".join([i for i in x if i.isdigit()])))

maps = test_data['StimuliName'].unique().tolist()
maps.sort()

# dropdown menu to select a stimuli map
select_map = Select (
    title = "Select map:", 
    value = "All",
    options = maps
)


# dropdown menu to select a user
select_user = Select (
    title = "Select user:", 
    value = "All",
    options = users
)


# slider for timestamp
slider_timestamp = Slider(
    start = test_data.Timestamp.min(), 
    end = test_data.Timestamp.max(), 
    step = 1,
    value = test_data.Timestamp.min(),
    title = 'Slide for time forward'
)


In [7]:
# function select the needed gprah according to the dropdown values

def select_graphs():
    stimulimap_val = select_map.value
    userselect_val = select_user.value.strip()
    
    selected = test_data.copy()
   
    if (stimulimap_val != "All"):
        selected = selected[selected['StimuliName'].str.contains(stimulimap_val)==True]
        users = selected['user'].unique().tolist()
        users = sorted(users, key=lambda x: int("".join([i for i in x if i.isdigit()])))
        select_user.options = users
    if (userselect_val != "All"):
        selected = selected[selected['user'] == userselect_val]
    
    return selected

# update the plot

def update():
    df = select_graphs()
    
    source.data = dict(
        Timestamp =  df['Timestamp'],
        StimuliName =  df['StimuliName'],
        FixationIndex =  df['FixationIndex'],
        FixationDuration =  df['FixationDuration'],
        MappedFixationPointX =  df['MappedFixationPointX'],
        MappedFixationPointY =  df['MappedFixationPointY'],
        user =  df['user']
        )

In [8]:
controls = [select_map, select_user, slider_timestamp]

for control in controls :
    control.on_change('value', lambda attr, old, new: update())

In [9]:
# scatter plot //  gaze plot
TOOLTIPS=[
    ("Time", "@Timestamp"),
    ("User", "@user"),
    ("X-coordinate", "@MappedFixationPointX"),
    ("Y-coordinate", "@MappedFixationPointY"),
]

labels = LabelSet(x = 'MappedFixationPointX',
                  y = 'MappedFixationPointY',
                  text = 'FixationIndex',
                  level = 'glyph',
                  x_offset = 0,
                  y_offset = 0,
                  source = source,
                  render_mode = 'canvas')

gaze_plot = figure (plot_width = 1000, plot_height = 650, title = 'test gaze plot', tooltips = TOOLTIPS, x_axis_label = 'x', y_axis_label = 'y')
gaze_plot.circle (x = 'MappedFixationPointX',
                  y = 'MappedFixationPointY',
                  source = source,
                  color = {'field' : 'user', 'transform' : color_mapper},
                  size = {'field' : 'FixationDuration', 'transform' : size_mapper},
                  fill_alpha = 0.5,
                 )
gaze_plot.line (x = 'MappedFixationPointX',
                y = 'MappedFixationPointY',
                source = source,
                color = 'purple',
                line_width = 2)


gaze_plot.y_range.flipped = True
gaze_plot.add_layout(labels)

AttributeError: unexpected attribute 'legend_label' to Circle, possible attributes are angle, angle_units, fill_alpha, fill_color, js_event_callbacks, js_property_callbacks, line_alpha, line_cap, line_color, line_dash, line_dash_offset, line_join, line_width, name, radius, radius_dimension, radius_units, size, subscribed_events, tags, x or y

In [ ]:
layout = row (
    gaze_plot, 
    column (select_map, select_user)
)

update()

curdoc().add_root(layout)

show(layout)
